<a href="https://colab.research.google.com/github/GuillaumeLe/pogchampnet/blob/master/PogchampNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementation of the PogchampNET

## Installation of dependencies


In [48]:
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
! pip install moviepy pytube
! imageio_download_bin ffmpeg
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from IPython.display import HTML, display
import cv2
import numpy as np
from keras.models import load_model
from pytube import YouTube
from google.colab import files
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

Ascertaining binaries for: ffmpeg.
Found GPU at: /device:GPU:0


You must provide a youtube URL

In [49]:
youtube_url = input('Enter the youtube URL of the video you want to use : ')
!rm -rf *.mp4
YouTube(youtube_url).streams.first().download('./')
!mv *.mp4 video.mp4
!ls -l

Enter the youtube URL of the video you want to use : https://www.youtube.com/watch?v=56PbKMl3ZVM&t=199s
total 937160
drwx------ 3 root root      4096 Nov 29 15:31 drive
drwxr-xr-x 2 root root      4096 Nov 20 18:17 sample_data
-rw-r--r-- 1 root root 654372080 Nov 29 16:17 saved_model.h5
-rw-r--r-- 1 root root 305260012 Nov 29 17:11 video.mp4


## Loading trained neural network

The farza's neural network given in the following link [Medium](https://medium.com/@farzatv/pogchampnet-how-we-used-twitch-chat-deep-learning-to-create-automatic-game-highlights-with-only-61ed7f7b22d4)

As the model is quite heavy it could take a long time

In [50]:
!wget https://www.dropbox.com/s/mlmr4qj9der7wdw/saved_model.h5
model = load_model('saved_model.h5')

--2018-11-29 17:12:12--  https://www.dropbox.com/s/mlmr4qj9der7wdw/saved_model.h5
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/mlmr4qj9der7wdw/saved_model.h5 [following]
--2018-11-29 17:12:12--  https://www.dropbox.com/s/raw/mlmr4qj9der7wdw/saved_model.h5
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc743cfb3e3e228ffc5527d948a4.dl.dropboxusercontent.com/cd/0/inline/AWiANYLR8egJsq4-9450TIxArN1SWXBaLIuitbi4rkzaAPl0zyJQhmWYeLb2AMF_J-kEWVYPfETALDmErH-yacS3uBYFP1Dd4ZxG7oCE6f2rK5dDfCxuSi-H8C0MZ_z8QhuzF0yCmzxhzFvW-ueIUBnxrleLF4RXEJ3nZmjVKULHEBM3ybcQtlYG321KuEpfIOE/file [following]
--2018-11-29 17:12:13--  https://uc743cfb3e3e228ffc5527d948a4.dl.dropboxusercontent.com/cd/0/inline/AWiANYLR8egJsq4-9450TIxArN1SWXBaLIuitbi4rk

# Prediction function

We define the way frame will be evaluated from 0 to 9

In [0]:
def predict(frame):
    frame = cv2.resize(frame, (384, 216))
    # dividing by 255 leads to faster convergence through normalization.
    frame = np.array(frame)/(255)
    frame = np.expand_dims(frame, axis=0)
    r = model.predict(frame)
#     print(np.argmax(r[0]), r[0][np.argmax(r[0])] * 100)
    return np.argmax(r[0])


Let's evaluate the score for each frame of the video. This task is the longer one it will analyse every frame and it take long time. A progress bar indicate the status above the following cell.

In [52]:
path_to_clip = './video.mp4'

video = cv2.VideoCapture(path_to_clip)
success, frame = video.read()
fps = int(video.get(cv2.CAP_PROP_FPS))
# video.set(1, 4500)
success, frame = video.read()
scores = []
frames = []
# plt.imshow(frame)
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
out = display(progress(0, length), display_id=True)
while success:
    success, frame = video.read()
    if not success:
        break

    frame_num = int(video.get(1))
    out.update(progress(frame_num, length))
    if frame_num % 10 != 0:
        continue

    r = predict(frame)
    scores.append(r)
    frames.append(video.get(cv2.CAP_PROP_POS_MSEC)//1000)

    if cv2.waitKey(0) == ord('a'):
        break
video.release()


# Register high scored sequences

Now we register the high scored sequences in your google drive as `highlightx.mp4` where `x` is the timestamp of the interesting sequence.

In [53]:
interest_index = []
window_size = 10 
last_index = 0
for index in range(len(scores[:-window_size])):
  if sum(scores[index:index+window_size])/window_size > 6:
    if len(interest_index) == 0 or index-1 != last_index:
      interest_index.append(index)
    last_index = index
print('Neural network found ', len(interest_index), 'sequence(s)')

Neural network found  1 sequence(s)


In [54]:
for index in interest_index:
  ffmpeg_extract_subclip(path_to_clip, max(0,frames[index]-5), frames[index]+25, targetname="./highlight"+str(frames[index])+".mp4")


[MoviePy] Running:
>>> /root/.imageio/ffmpeg/ffmpeg-linux64-v3.3.1 -y -i ./video.mp4 -ss 324.00 -t 30.00 -vcodec copy -acodec copy ./highlight329.0.mp4
... command successful.


# Download hightlight


In [55]:
import io
import base64
from IPython.display import HTML
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('./') if isfile(join('./', f))]
data = ''

for file in onlyfiles:
  if file.startswith('highlight'):
    print(file)
    files.download(file)
    

highlight329.0.mp4
